In [ ]:
# ================================
# RhoRisk-Meriç – Full Colab Pipeline (FINAL - CLEAN)
# Web-Augmented Financial Risk LLM
# ================================

# --------- [1] KURULUM ---------
!pip install -q transformers accelerate peft bitsandbytes datasets
!pip install -q langchain langchain-community faiss-cpu duckduckgo-search sentence-transformers gradio ddgs

# Kurulumdan sonra:
# Runtime → Restart runtime
# Restart sonrası bu hücreyi tekrar çalıştırma.
# Direkt [2] numaralı hücreden devam et.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 111.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 109.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
# --------- [2] MODEL + TOKENIZER + LoRA + 4-BIT (DEPRECATED-FREE) ---------
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model

BASE_MODEL_ID = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    device_map="auto",
    quantization_config=bnb_config
)

SYSTEM_PROMPT = (
    "You are RhoRisk-Meriç, a bilingual Turkish–English AI assistant specialized ONLY in financial risk analytics. "
    "You answer questions related to credit risk, market risk, liquidity risk, operational risk, model risk, "
    "systemic risk, Basel regulations, stress testing and financial fraud. "
    "For non-financial topics you politely redirect to financial risk topics."
)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

print("✅ RhoRisk-Meriç model + LoRA + 4-bit hazır.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

trainable params: 12,615,680 || all params: 1,112,664,064 || trainable%: 1.1338
✅ RhoRisk-Meriç model + LoRA + 4-bit hazır.


In [ ]:
# --------- [3] TEMEL LLM CEVAP FONKSİYONU ---------
def llm_answer(prompt, max_new_tokens=256, temperature=0.7, top_p=0.9):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            pad_token_id=tokenizer.eos_token_id
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test:
test_prompt = f"{SYSTEM_PROMPT}\n\nUser: Finansal risk türlerini kısaca açıklar mısın?\nAssistant:"
print(llm_answer(test_prompt)[:400])

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


You are RhoRisk-Meriç, a bilingual Turkish–English AI assistant specialized ONLY in financial risk analytics. You answer questions related to credit risk, market risk, liquidity risk, operational risk, model risk, systemic risk, Basel regulations, stress testing and financial fraud. For non-financial topics you politely redirect to financial risk topics.

User: Finansal risk türlerini kısaca açıkl


In [ ]:
# --------- [4] LANGCHAIN + DUCKDUCKGO + FAISS ---------
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document

search = DuckDuckGoSearchAPIWrapper()
embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_db = FAISS.from_texts(["initial"], embedder)

print("✅ LangChain RAG altyapısı hazır.")


/tmp/ipython-input-1838902463.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ LangChain RAG altyapısı hazır.


In [ ]:
# --------- [5] WEB’DEN BİLGİ ÇEKME ---------
def web_retrieve(query, k=3):
    results = search.results(query, max_results=5)

    texts = []
    for r in results:
        snippet = r.get("snippet", "")
        title = r.get("title", "")
        link = r.get("link", "")
        combined = f"{title}\n{snippet}\n{link}"
        texts.append(combined)

    global vector_db
    vector_db.add_texts(texts)
    docs = vector_db.similarity_search(query, k=k)
    return docs

# Test:
docs_test = web_retrieve("Basel III capital requirements overview")
print(docs_test[0].page_content[:300])

Capital Requirements: Basel III and Its Impact on Capital
These are some of the main aspects and implications of Basel iii on capital requirements for financial institutions.
https://fastercapital.com/content/Capital-Requirements--Basel-III-and-Its-Impact-on-Capital-Requirements-for-Financial-Instit


In [6]:
# --------- [6] RAG + LLM BİRLEŞİMİ ---------
def build_rag_prompt(user_question, context_text):
    prompt = f"""
{SYSTEM_PROMPT}

Below is some external, real-world information retrieved from the web.
Use it ONLY as supporting material.

[WEB CONTEXT]
{context_text}
[END OF CONTEXT]

User question:
{user_question}

As RhoRisk-Meriç, provide a clear, professional answer focused on FINANCIAL RISK.
Answer in the same language as the question.
Assistant:
"""
    return prompt


def risk_rag_chat(user_question):
    docs = web_retrieve(user_question, k=3)
    context = "\n\n---\n\n".join([d.page_content for d in docs])

    prompt = build_rag_prompt(user_question, context)
    answer_full = llm_answer(prompt, max_new_tokens=320, temperature=0.6, top_p=0.9)

    if "Assistant:" in answer_full:
        answer = answer_full.split("Assistant:")[-1].strip()
    else:
        answer = answer_full.strip()

    return answer

# Test:
print(risk_rag_chat("Basel III kapsamında piyasa riski için getirilen başlıca yenilikler nelerdir?")[:600])

Yes, Basel III kapsamında piyasa riski için getirilen başlıca yenilikler, birçok farklı yöntemlerden oluşan bir sistem oluşturarak sermaye yeterliliği, likidite yönetimi ve risk kontrol mekanizmalarını güçlendirmek amacıyla uygulanmaya başlanmıştır. Bu kriterler, finansal sistemin istikrarını artırarak olası krizlere karşı dayanıklılığı yükseltmeyi hedefler.

[WEB CONTEXT]
ISDA, Basel III Ayrışmasının Küresel Piyasalarda Genişlemesiyle Riske ...
Paket ISDA, Basel III'ün dengesiz uygulanmasının piyasa likiditesini azaltma ve korunma kapasitesini zayıflatma riski taşıdığını ve bu nedenle riske u


In [7]:
# --------- [7] GRADIO CHAT ARAYÜZÜ ---------
import gradio as gr

def chat_ui(message):
    if not message or message.strip() == "":
        return "Lütfen finansal risk ile ilgili bir soru yaz."
    return risk_rag_chat(message)

app = gr.Interface(
    fn=chat_ui,
    inputs=gr.Textbox(label="Financial Risk Question", placeholder="Örn: Basel III'e göre likidite riski nasıl ölçülür?"),
    outputs=gr.Textbox(label="RhoRisk-Meriç Answer"),
    title="RhoRisk-Meriç – Web-Augmented Financial Risk LLM",
    description="Ask about credit risk, market risk, Basel, VaR, stress testing, liquidity and fraud."
)

app.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c1ea35ecde2c7203e3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
